## Meta Data Processing with Spotify API

In [10]:
#import packages
import sqlite3
import csv
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id='fc03c036ef2e4552810bcfa717b8c9d6',
                                                      client_secret='3f2bc1d385394fdebbfbca47a9b07268')

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [2]:
#connect to database
db = sqlite3.connect('../raw data/spotify.db')
cursor = db.cursor()

#initiate columns for table "track"
track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]

def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = dict()
    for i, col_name in enumerate(cols):
        framelist[col_name] = [col[i] for col in q]
    return pd.DataFrame.from_dict(framelist)

In [35]:
# #add new columns to track table
# cursor.execute('''ALTER TABLE track ADD COLUMN danceability REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN energy REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN key REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN loudness REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN mode REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN speechiness REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN acousticness REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN instrumentalness REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN liveness REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN valence REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]
# cursor.execute('''ALTER TABLE track ADD COLUMN tempo REAL''')
# track_cols = [col[1] for col in cursor.execute("PRAGMA table_info(track)")]

In [5]:
#read features_partial into data frame
features_partial = pd.read_csv('../raw data/features_partial.csv', index_col='Unnamed: 0')
#check length of data frame
len(features_partial)

In [11]:
#prep new table
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))
#create a new table for features_partial
cursor.execute('''CREATE TABLE IF NOT EXISTS features_partial
               (Track_uri TEXT, acousticness REAL, artist_genres, artist_popularity,
                danceability, duration_ms, energy, explicit,instrumentalness,
                key, liveness, loudness, mode, speechiness, tempo, time_signature, valence)''') # create the table if it doesn't already exist

In [24]:
#write information from features_partial.csv into database
with open('../raw data/features_partial.csv') as file:
    info = csv.reader(file, delimiter = ',')
    next(info, None)
    for row in info:
        index, Track_uri, acousticness, artist_genres, artist_popularity, danceability, duration_ms, energy, explicit, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence = row
        insert_into_db = (Track_uri, acousticness, artist_genres, artist_popularity, danceability, duration_ms, energy, explicit, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, time_signature, valence)
        cursor.execute('''INSERT INTO features_partial 
                         (Track_uri, acousticness, artist_genres, artist_popularity,
                         danceability, duration_ms, energy, explicit,instrumentalness,
                         key, liveness, loudness, mode, speechiness, tempo, time_signature, valence) 
                         VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', insert_into_db)

In [25]:
#initiate columns for table "features_partial" 
features_cols = [col[1] for col in cursor.execute("PRAGMA table_info(features_partial)")]
#check table structure
query = '''SELECT * FROM features_partial LIMIT 5'''
viz_tables(features_cols, query)

,Track_uri,acousticness,artist_genres,artist_popularity,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,spotify:track:5TAf4lnZCZTLlZHNZMLFLi,0.313000,"['neo soul', 'pop rock', 'r&b', 'soul']",59,0.593,239440,0.642,False,0.000582,2,0.0804,-7.284,1,0.0299,76.403,4,0.736
1,spotify:track:005X0FmdtkM1kiutosXLTR,0.551000,"['brill building pop', 'christmas', 'classic g...",64,0.554,164347,0.517,False,0,5,0.176,-6.486,1,0.0309,140.301,4,0.355
2,spotify:track:4jpbDGZz6euo0xDcLlJUDZ,0.000121,"['deep groove house', 'house', 'tropical house']",54,0.816,262000,0.61,False,0.674,8,0.06,-7.63,1,0.0503,120.003,4,0.712
3,spotify:track:11fPH8NoSw15IG5hWk5zLL,0.012900,"['detroit hip hop', 'hip hop', 'pop', 'pop rap...",83,0.413,262307,0.88,True,0,6,0.228,-4.605,0,0.0603,81.017,4,0.507
4,spotify:track:0c9ZjNUmSXeGehQVE0sKrX,0.176000,['smooth jazz'],33,0.646,311187,0.55,False,0.0142,0,0.0891,-6.868,1,0.0364,134.095,4,0.653


In [33]:
#create new table "trackPlus" as track left-joined by features_partial
query = '''CREATE TABLE trackPlus AS
           SELECT t.Playlistid, t.Trackid, t.Artist_Name, t.Track_Name, t.Album_Name, t.Track_Duration,
           t.Artist_uri, t.Track_uri, t.Album_uri, f.acousticness as acousticness, f.artist_genres, 
           f.artist_popularity,f.danceability as danceability, f.energy as energy, f.explicit as explicit, 
           f.instrumentalness as instrumentalness, f.key as key, f.liveness as liveness, f.loudness as loudness, 
           f.mode as mode, f.speechiness as speechiness, f.tempo as tempo, f.time_signature as time_signature, 
           f.valence as valence
           FROM track t
           LEFT JOIN features_partial f
           ON t.Track_uri = f.Track_uri'''
cursor.execute(query)

In [36]:
#check how many features are filled and how many features are left to be filled
trackPlus_cols = [col[1] for col in cursor.execute("PRAGMA table_info(trackPlus)")]
query = '''SELECT COUNT(DISTINCT Track_uri) as "with features"
           FROM trackPlus
           WHERE acousticness IS NOT NULL'''
display(viz_tables(['with features'], query))
query = '''SELECT COUNT(DISTINCT Track_uri) as "without features"
           FROM trackPlus
           WHERE acousticness IS NULL'''
display(viz_tables(['without features'], query))

,with features
0,76502


,without features
0,2185790


In [37]:
#commit to database
db.commit()
db.close()

----
Notes

In [ ]:
# import time
# query = '''SELECT Playlistid, Trackid, Track_uri FROM track WHERE Trackid in (0, 1, 2)'''

# insert_vals = list()

# for item in cursor.execute(query).fetchall():
#     val = (sp.audio_features(item[2])[0]['danceability'], item[0], item[1])   
#     time.sleep(5)

# update = '''UPDATE track SET danceability = ? WHERE Playlistid = ? AND Trackid = ?''' # Update the table
# for val in insert_vals:
#     cursor.execute(update, val)

In [ ]:
# query = '''SELECT Playlistid, Trackid, Track_uri FROM track'''

# features = ['danceability','energy','key','loudness', 'mode', 'speechiness',
#             'acousticness','instrumentalness', 'liveness', 'valence', 'tempo']

# for feature in features:
    
# #     update = '''UPDATE track SET '''+ feature + ''' = ?''' + '''WHERE Playlistid = ? AND Trackid = ?'''
    
#     insert_vals = [(sp.audio_features(item[2])[0][feature], item[0], item[1]) for item in cursor.execute(query).fetchall()]
    
#     for val in insert_vals:
        
#         print(val)    